<a href="https://colab.research.google.com/github/Ekliipce/Representation-and-Generative-Learning/blob/main/Reimpl%C3%A9mentation_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RNN Reimplementation


We aim at writing a simple RNN that will be able to sum two binary strings.

Inputs will be successive bits that the network will have to add and the output is going to be the result bit.

In such a case, we obviously see why a RNN might work where a simple NN could not, because of the result of the previous addition needs to be kept in memory.

\begin{matrix}
& 1 & 0 & 1 & 1 \\
+ & 0 & 0 & 0 & 1 \\
= & 1 & 1 & 0 & 0 \\
\end{matrix}

In [3]:
import copy
import numpy as np
from tqdm import tqdm

In [4]:
def sigmoid(x):
    """
    Le calcul de la fonction sigmoid.
    """
    return 1 / (1 + np.exp(-x))


def d_sigmoid(x):
    """
    Le calcul de la déCopie de TPrivée de la fonction sigmoid.
    """
    sig = sigmoid(x)
    return sig * (1 - sig)


def d_sigmoid_easy(x):
    """
    Une variation de la dérivée de la sigmoid, pour simplifier certains calcul
    lors de la rétropropagation.
    """
    return x * (1 - x)

In [5]:
def generate_dataset(d):
    """
    Génère le jeu de données au format dictionnaire :
    clef = nombre entier
    valeur = tableau binaire correspondant (big endian)

    :param d: the dimension of one bits string
    :return: the dataset
    """
    dataset = {}
    largest_number = pow(2, d)
    binary = np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T, axis=1)
    for i, b in enumerate(binary):
        dataset[i] = b
    return dataset


def get_sample(dataset):
    """
    Sélectionne deux nombres a et b dans l'ensemble de données et calcule la somme c.
    Si la somme est inférieure au nombre le plus grand du dictionnaire, renvoie :
    le tableau de bits de a, le tableau de bits de b, le tableau de bits de c
    -> dataset[a], dataset[b], dataset[c]
    """
    choices = list(dataset.keys())
    max = choices[-1]

    while(True):
      a = int(np.random.choice(choices))
      b = int(np.random.choice(choices))
      c = a + b
      if (c < max):
        bit_a = dataset[a]
        bit_b = dataset[b]
        bit_c = dataset[c]

        return bit_a, bit_b, bit_c

In [ ]:
dataset = generate_dataset(10)
get_sample(dataset)

(array([0, 1, 1, 1, 1, 0, 1, 1], dtype=uint8),
 array([1, 1, 0, 1, 1, 1, 0, 1], dtype=uint8),
 array([0, 1, 0, 1, 1, 0, 0, 0], dtype=uint8))

In [6]:
def init_nn(inp_dim, hid_dim, out_dim):
    """
    Initialise les paramètres du réseaux : wi, wh, wo ainsi que les tenseurs
    des mêmes tailles qui serviront pour les mettre à jour.

    wi, wh, wo sont initialisés aléatoirement suivant une loi uniforme allant
    de -1 à 1.

    Utiliser la fonction numpy.random.random.

    wi_update, wh_update, wo_update ont la même taille que wi, wh et wo
    respectivement mais ne contiennent à l'initialisation que des 0.

    Utiliser soit :
    - np.zeros -> prend une taille en argument
    - np.zeros_like -> prend un tenseur en argument

    :param inp_dim: dimension de l'entrée
    :param hid_dim: dimension de la couche cachée
    :param out_dim: dimension de la sortie
    :return: les paramètres
    """
    wi_layer = np.random.random((hid_dim, inp_dim))
    wh_layer = np.random.random((hid_dim, hid_dim))
    wo_layer = np.random.random((out_dim, hid_dim))
    return wi_layer, wh_layer, wo_layer

## Forward
$$a_t = w_ix_t + w_hh_{t-1}$$

$$h_t = σ(a_t)$$

$$o_t = w_oh_t$$

$$ŷ = \sigma(o_t)$$



## Backward
With $L$, loss function as $\sum^N_{i=0}(ŷ - y)²$ : <br/><br/>

1. $$\frac{∂L}{\partial w_o} = \frac{∂L_t}{\partial o_t} \frac{∂o_t}{\partial w_o} = \frac{∂L_t}{\partial o_t} \frac{∂w_oh_t}{\partial w_o} = \frac{∂L_t}{\partial o_t} h_t$$
and $$\frac{∂L}{\partial o_t} = \frac{∂L_t}{\partial ŷ_t}\frac{∂ŷ_t}{\partial o_t} = \frac{∂(ŷ - y)²}{\partial ŷ_t}\frac{∂σ(o_t)}{\partial o_t} = 2(ŷ-y)*ŷ(1-ŷ)$$
so $$\frac{∂L}{\partial w_o} = 2(ŷ-y)*ŷ(1-ŷ) * h_t$$

___
<br/><br/>

2. $$\frac{\delta L}{\delta w_i} = \frac{\delta L}{\delta h_t}\frac{\delta h_t}{\delta a_t}\frac{\delta a_t}{\delta w_i} = \frac{\delta L}{\delta h_t}h_t(1-h_t)x_t$$

<br/><br/>
___

<br/><br/>

3. $$\frac{\delta L}{\delta w_h} = \frac{\delta L}{\delta h_t} . \frac{\delta h_t}{\delta a_t}.\frac{\delta a_t}{\delta w_h} = h_{t-1}h_t(1 - h_t)\frac{\delta L}{\delta h_t}$$

<br/><br/>

___

with

$$\frac{\delta L}{\delta h_t} = \frac{\delta L_t}{\delta h_t} + \frac{\delta L_{i > t}}{\delta h_t} = \frac{\delta L_t}{\delta o_t}.\frac{\delta o_t}{\delta h_t} + \frac{\delta L}{\delta h_{t+1}}.\frac{\delta h_{t+1}}{\delta a_{t+1}}.\frac{\delta a_{t+1}}{\delta h_t}$$

$$\frac{\delta L}{\delta h_t} = w_o\frac{\delta L_t}{\delta o_t} + w_h h_{t+1}(1 - h_{t+1})\frac{\delta L} {\delta h_{t+1}}$$


In [21]:
def train(wi, wh, wo, iterations, dataset):
    for i in range(iterations):
        a, b, c = get_sample(dataset)
        d = np.zeros_like(c)  # stores our successive predictions
        error = 0
        inputs = []
        truth = []
        o_deltas = []  # stores gradient of error relative to ot
        ht_values = []  # stores values of hidden states
        ht_values.append(np.zeros((hidden_dimension, 1)))

        # Forward
        for pos in range(number_bits):
            """
            1. On récupère les premiers bits de a et de b -> devient notre x
            2. On récupère le premier bit de c -> devient notre y

            """
            input = np.array([a[pos], b[pos]]).reshape(-1, 1)
            y = np.array([c[pos]])

            inputs.append(input)
            truth.append(y)

            # Compute a_t
            a_t = wi @ input + wh @ ht_values[pos]

            # Compute h_t
            h_t = sigmoid(a_t)
            ht_values.append(h_t)

            # Compute o_t
            o_t = wo @ h_t

            # Compute y_pred
            y_pred = sigmoid(o_t)

            d[pos] = np.around(y_pred)


            o_delta = (y_pred - y) * d_sigmoid(y_pred)
            o_deltas.append(o_delta)


        future_ht_delta = np.zeros_like(ht_values[-1])  # stores delta L / delta ht+1, necessary when computing delta L / delta ht+1
        future_ht = np.zeros_like(ht_values[-1])  # stores ht+1, 0 when we are at last t

        # Backward

        wi_update = np.zeros_like(wi)
        wh_update = np.zeros_like(wh)
        wo_update = np.zeros_like(wo)


        for pos in reversed(range(number_bits)):
            """
            1. On récupère le x en partant de la fin de la séquence (le premier bit)
            2. On récupère la valeur de son état caché ht dans la liste
            3. On récupère la valeur de l'état caché précédent dans la liste ht-1
            4. On récupère la valeur de delta L / delta ot dans la liste
            5. On calcule ht_delta (delta L / delta ht) -> partie la plus complexe
            6. On calcule wo_update, wh_update, wi_update (gradients pour wo, wh, wi)
            """
            # Get first bit
            x_i = inputs[pos]
            y_pred = d[pos]
            y = truth[pos]

            # Get h_t
            h_t = ht_values[pos + 1]


            # Get h_{t-1}
            h_t_prev = ht_values[pos]

            # Get partial derivative L with respect to o_t
            o_delta = o_deltas[pos]

            # Get partial derivative L with respect to h_t
            h_delta = (wo.T @ o_delta + future_ht_delta) * d_sigmoid(h_t)

            # Compute partials derivative L with respect to weights
            wo_update += np.outer(o_delta, h_t)
            wh_update += np.outer(h_delta, h_t_prev)
            wi_update += np.outer(h_delta, x_i)

            # Update values
            h_delta_next = h_delta
            h_t_next = h_t

        wi -= 0.001 * wi_update
        wh -= 0.001 * wh_update
        wo -= 0.001 * wo_update

        if i % 1000 == 0:
            print(f"[{i}|{iterations}] Error: {error}")
            print(f"\tTrue: {a} + {b} = {c}")
            print(f"\tPred: {d}")

In [22]:
alpha = 0.1
input_dimension = 2
hidden_dimension = 16
output_dimension = 1
number_bits = 8
data = generate_dataset(number_bits)
nn = init_nn(input_dimension, hidden_dimension, output_dimension)
train(*nn, 10000, data)

[0|10000] Error: 0
	True: [0 1 0 0 1 1 1 1] + [0 1 0 1 1 0 1 0] = [1 0 1 0 1 0 0 1]
	Pred: [1 1 1 1 1 1 1 1]
[1000|10000] Error: 0
	True: [0 0 0 1 0 0 0 0] + [1 1 0 0 0 1 0 1] = [1 1 0 1 0 1 0 1]
	Pred: [1 1 1 1 1 1 1 1]
[2000|10000] Error: 0
	True: [0 0 0 0 0 1 1 1] + [0 1 0 0 1 1 0 1] = [0 1 0 1 0 1 0 0]
	Pred: [1 1 1 1 1 1 1 1]
[3000|10000] Error: 0
	True: [0 0 0 0 0 1 0 0] + [0 0 1 0 0 0 0 0] = [0 0 1 0 0 1 0 0]
	Pred: [1 1 1 1 1 1 1 1]
[4000|10000] Error: 0
	True: [0 1 0 1 1 0 1 0] + [0 0 0 1 0 1 1 1] = [0 1 1 1 0 0 0 1]
	Pred: [1 1 1 1 1 1 1 1]
[5000|10000] Error: 0
	True: [1 0 1 0 1 0 1 0] + [0 0 0 1 0 0 1 0] = [1 0 1 1 1 1 0 0]
	Pred: [1 1 1 1 1 1 1 1]
[6000|10000] Error: 0
	True: [1 1 0 1 1 1 0 1] + [0 0 0 0 0 0 0 0] = [1 1 0 1 1 1 0 1]
	Pred: [1 1 1 1 1 1 1 1]
[7000|10000] Error: 0
	True: [0 0 1 1 1 1 1 1] + [1 0 1 0 1 1 1 0] = [1 1 1 0 1 1 0 1]
	Pred: [1 1 1 1 1 1 1 1]
[8000|10000] Error: 0
	True: [0 0 0 1 0 0 1 1] + [1 1 0 1 0 1 1 1] = [1 1 1 0 1 0 1 0]
	Pred: [1 1 1 1 1 1 